<a href="https://colab.research.google.com/github/PhaneendraYedidi/Machine-Learning-Practices/blob/master/UnitPricesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV 


In [62]:
#Mounting the Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
#Reading the train and test data from drive
pathExt = 'drive/MyDrive/HackathonData/'
trainData = pd.read_csv(pathExt + "Train.csv")
testData = pd.read_csv(pathExt + "Test.csv")

Business Constraints:

*   Multivariate Regression
*   Big dataset, underfitting vs overfitting
*   Optimizing RMSE to generalize well on unseen data






#Data Cleaning

In [64]:
trainData.head()
#trainData.value_counts()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [65]:
testData.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country
0,3139,1709,1079,3,2011-02-22 15:22:00,16161.0,35
1,617,510,3457,1,2010-12-08 12:46:00,17341.0,35
2,14653,604,694,36,2011-10-25 13:53:00,15158.0,35
3,8634,1478,3473,2,2011-06-27 12:38:00,16033.0,35
4,15546,3216,871,1,2011-11-06 16:14:00,15351.0,35


In [66]:
trainData.describe()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
count,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000
mean,9955.394083,1573.386807,2023.955573,12.028468,3.451216,15288.224278,32.823846
std,5551.519138,843.604991,1089.812078,295.111588,78.399541,1714.336529,6.743602
min,0.000000,0.000000,0.000000,-80995.000000,0.000000,12346.000000,0.000000
25%,5069.000000,939.000000,1141.000000,2.000000,1.250000,13953.000000,35.000000
50%,10310.000000,1521.000000,1987.000000,5.000000,1.950000,15152.000000,35.000000
75%,14657.000000,2106.000000,2945.000000,12.000000,3.750000,16794.000000,35.000000
max,22188.000000,3683.000000,3895.000000,80995.000000,38970.000000,18287.000000,36.000000


In [67]:
#Min value of Quantity is negative which is not realistic so removing the values
print('Shape of train data',trainData.shape)
trainData = trainData[trainData['Quantity']>0]
print('Shape of train data',trainData.shape)

Shape of train data (284780, 8)
Shape of train data (278613, 8)


In [68]:
trainData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278613 entries, 0 to 284779
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    278613 non-null  int64  
 1   StockCode    278613 non-null  int64  
 2   Description  278613 non-null  int64  
 3   Quantity     278613 non-null  int64  
 4   InvoiceDate  278613 non-null  object 
 5   UnitPrice    278613 non-null  float64
 6   CustomerID   278613 non-null  float64
 7   Country      278613 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 19.1+ MB


No Null values in the Training data

In [69]:
'''
s = sum(trainData.Quantity.values);
h = trainData.Quantity.values/s;
plt.plot(h, label="Histrogram of Quantity")
plt.xlabel('Index of a Quantity')
plt.ylabel('Number of Occurances')
plt.legend()
plt.grid()
plt.show()

c = np.cumsum(h)
plt.plot(c,label='Cumulative distribution of Quantity')
plt.grid()
plt.legend()
plt.show()
'''

'\ns = sum(trainData.Quantity.values);\nh = trainData.Quantity.values/s;\nplt.plot(h, label="Histrogram of Quantity")\nplt.xlabel(\'Index of a Quantity\')\nplt.ylabel(\'Number of Occurances\')\nplt.legend()\nplt.grid()\nplt.show()\n\nc = np.cumsum(h)\nplt.plot(c,label=\'Cumulative distribution of Quantity\')\nplt.grid()\nplt.legend()\nplt.show()\n'

In [70]:
'''
plt.figure(figsize=(20, 10))

plt.hist(trainData.Quantity.value_counts(), bins=160)

#plt.yscale('log', nonposy='clip')

plt.title('Histogram of quantity')
'''

"\nplt.figure(figsize=(20, 10))\n\nplt.hist(trainData.Quantity.value_counts(), bins=160)\n\n#plt.yscale('log', nonposy='clip')\n\nplt.title('Histogram of quantity')\n"

In [71]:
for i in np.arange(10,101,10):
  p = np.percentile(trainData.Quantity, i)
  print('Percentile ', str(i), "is ", str(p))

Percentile  10 is  1.0
Percentile  20 is  2.0
Percentile  30 is  2.0
Percentile  40 is  4.0
Percentile  50 is  6.0
Percentile  60 is  8.0
Percentile  70 is  12.0
Percentile  80 is  12.0
Percentile  90 is  24.0
Percentile  100 is  80995.0


In [72]:
for i in np.arange(90,101,1):
  p = np.percentile(trainData.Quantity, i)
  print('Percentile ', str(i), "is ", str(p))

Percentile  90 is  24.0
Percentile  91 is  24.0
Percentile  92 is  24.0
Percentile  93 is  25.0
Percentile  94 is  32.0
Percentile  95 is  36.0
Percentile  96 is  48.0
Percentile  97 is  50.0
Percentile  98 is  96.0
Percentile  99 is  120.0
Percentile  100 is  80995.0


In [73]:
for i in np.arange(99,100.1,.1):
  p = np.percentile(trainData.Quantity, i)
  print('Percentile ', str(i), "is ", str(p))

Percentile  99.0 is  120.0
Percentile  99.1 is  144.0
Percentile  99.19999999999999 is  144.0
Percentile  99.29999999999998 is  144.0
Percentile  99.39999999999998 is  192.0
Percentile  99.49999999999997 is  200.0
Percentile  99.59999999999997 is  216.0
Percentile  99.69999999999996 is  256.0
Percentile  99.79999999999995 is  320.0
Percentile  99.89999999999995 is  500.0
Percentile  99.99999999999994 is  80994.99999881606


In [74]:
#There is a big change in percentile of quantity from 99.89. This can be thought of as outlier
print('Shape of train data',trainData.shape)
#trainData = trainData[trainData['Quantity']>=500]
trainData = trainData[trainData.Quantity < np.percentile(trainData.Quantity,99.9)]
print('Shape of train data',trainData.shape)

Shape of train data (278613, 8)
Shape of train data (278323, 8)


In [75]:
'''
for i in np.arange(10,101,10):
  p = np.percentile(trainData.UnitPrice, i)
  print('Percentile ', str(i), "is ", str(p))
'''

'\nfor i in np.arange(10,101,10):\n  p = np.percentile(trainData.UnitPrice, i)\n  print(\'Percentile \', str(i), "is ", str(p))\n'

In [76]:
#feature_cols = ["StockCode", "Quantity" ]
feature_cols = ["StockCode","Description", 	"Quantity",	"CustomerID",	"Country"]
x = trainData[feature_cols]
y = trainData["UnitPrice"]

In [77]:
# Split data into train and test format
#x_train, x_cv, y_train, y_cv = train_test_split(x, y, test_size=0.2, random_state=None)

#Linear Regression

In [78]:
'''
# Creating the hyperparameter grid  
param_grid = {"fit_intercept": [True, False]} 
  
# Instantiating model 
model = LinearRegression()
  
# Instantiating RandomizedSearchCV object 
rand_search = RandomizedSearchCV(model, param_grid, cv = 5) 
  
rand_search.fit(x, y)
print(rand_search.best_estimator_.fit_intercept)
model_best = LinearRegression(fit_intercept = rand_search.best_estimator_.fit_intercept)
model_best.fit(x, y)

y_pred_train_linReg = model_best.predict(x)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y,y_pred_train_linReg)))

x_test = testData[feature_cols]
y_pred_test_linReg = model_best.predict(x_test)
print(y_pred_test_linReg)
'''

'\n# Creating the hyperparameter grid  \nparam_grid = {"fit_intercept": [True, False]} \n  \n# Instantiating model \nmodel = LinearRegression()\n  \n# Instantiating RandomizedSearchCV object \nrand_search = RandomizedSearchCV(model, param_grid, cv = 5) \n  \nrand_search.fit(x, y)\nprint(rand_search.best_estimator_.fit_intercept)\nmodel_best = LinearRegression(fit_intercept = rand_search.best_estimator_.fit_intercept)\nmodel_best.fit(x, y)\n\ny_pred_train_linReg = model_best.predict(x)\nprint(\'RMSE:\', np.sqrt(metrics.mean_squared_error(y,y_pred_train_linReg)))\n\nx_test = testData[feature_cols]\ny_pred_test_linReg = model_best.predict(x_test)\nprint(y_pred_test_linReg)\n'

#Logistic Regression

In [79]:
'''
from sklearn.linear_model import LogisticRegression

# Creating the hyperparameter grid  
param_grid = {'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['liblinear']} 
  
# Instantiating model 
model = LogisticRegression()
  
# Instantiating RandomizedSearchCV object 
rand_search = RandomizedSearchCV(model, param_grid, cv = 5) 
  
rand_search.fit(x, y)
print(rand_search.best_estimator_)
model_best = LogisticRegression(penalty = rand_search.best_estimator_.penalty,
                                  C = rand_search.best_estimator_.C,
                                solver = rand_search.best_estimator_.solver)
model_best.fit(x, y)

y_pred_train_logReg = model_best.predict(x)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y,y_pred_train_logReg)))

y_pred_test_logReg = model_best.predict(x_test)
print(y_pred_test_logReg)
'''

"\nfrom sklearn.linear_model import LogisticRegression\n\n# Creating the hyperparameter grid  \nparam_grid = {'penalty' : ['l1', 'l2'],\n    'C' : np.logspace(-4, 4, 20),\n    'solver' : ['liblinear']} \n  \n# Instantiating model \nmodel = LogisticRegression()\n  \n# Instantiating RandomizedSearchCV object \nrand_search = RandomizedSearchCV(model, param_grid, cv = 5) \n  \nrand_search.fit(x, y)\nprint(rand_search.best_estimator_)\nmodel_best = LogisticRegression(penalty = rand_search.best_estimator_.penalty,\n                                  C = rand_search.best_estimator_.C,\n                                solver = rand_search.best_estimator_.solver)\nmodel_best.fit(x, y)\n\ny_pred_train_logReg = model_best.predict(x)\nprint('RMSE:', np.sqrt(metrics.mean_squared_error(y,y_pred_train_logReg)))\n\ny_pred_test_logReg = model_best.predict(x_test)\nprint(y_pred_test_logReg)\n"

#All Models

In [ ]:
from sklearn import linear_model
from sklearn import svm

classifiers = [
    svm.SVR(),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]

x_test = testData[feature_cols]
y_train_Pred = []
y_test_Pred = []

for item in classifiers:
    clf = item
    clf.fit(x, y)
    pred_train_data = clf.predict(x)
    y_train_Pred.append(pred_train_data)
    print('RMSE ',item,' :', np.sqrt(metrics.mean_squared_error(y,pred_train_data)))
    y_test_Pred.append(clf.predict(x_test))

print(y_test_Pred)
y_test_final = (y_test_Pred[0] + y_test_Pred[1])/2
print(y_test_final)

In [ ]:
prediction = pd.DataFrame(y_test_final, columns=['UnitPrice']).to_csv('UnitPricesPrediction.csv')


In [ ]:
'''
import numpy as np
from sklearn import linear_model
from sklearn import svm

classifiers = [
    svm.SVR(),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]

x_test = testData[feature_cols]

for item in classifiers:
    clf = item
    clf.fit(x, y)
    pred_data = clf.predict(x)
    print('RMSE ',item,' :', np.sqrt(metrics.mean_squared_error(y,pred_data)))
'''


Feature selection;

Normalizing Features;

Select Loss function and Hypothesis;

Set Hypothesis Parameters;

Minimize the Loss Function;

Test the hypothesis function

Linear Regression
Logistic Regression
Polynomial Regression
Stepwise Regression
Ridge Regression
Lasso Regression
ElasticNet Regression

https://www.listendata.com/2018/03/regression-analysis.html

https://www.listendata.com/2018/03/regression-analysis.html

https://stackoverflow.com/questions/41925157/logisticregression-unknown-label-type-continuous-using-sklearn-in-python

https://stackoverflow.com/questions/33091376/python-what-is-exactly-sklearn-pipeline-pipeline